#### This file contains quality check on the original dtseries, currently it only contains the temporal signal-to-noise ratio


In [1]:
import nibabel as nib
import numpy as np
import seaborn as sns
import pandas as pd
import scipy.io as sio
import time, os, pickle, json
import matplotlib.pyplot as plt
import matplotlib as mpl
from nod_utils import save_ciftifile
from os.path import join as pjoin
from matplotlib import font_manager
font_manager.fontManager.addfont('./supportfiles/arial.ttf')
# define plot utils
mpl.rcParams['axes.linewidth'] = 2
mpl.rcParams.update({'font.size': 12, 'font.family': 'Arial', 'mathtext.fontset': 'stix'})

In [2]:
# define calc tSNR function   
def tsnr(data):
    mean = data.mean(axis=-1)
    std = data.std(axis=-1)
    return np.nan_to_num(mean/std)

# define the time-series function
def get_task_time_series(sub_name, ciftify_dir, result_dir, task, n_run=None, roi_mat = None):
    # MNINolinear/Results disposit all the runs data
    _result_path = pjoin(ciftify_dir, sub_name, result_dir)
    # extract the task runs
    task_runs = [_ for _ in os.listdir(_result_path) if (task in _)]
    task_runs.sort() 
    # initialize the mapping dict
    stim_resp_map = {}
    # loop run
    if n_run :
        task_runs = task_runs[:n_run]
    for single_run in task_runs:
        print(single_run + ' in ' + sub_name)
        # prepare .feat/GrayordinatesStats dir
        nii_dir = '{0}'.format(single_run)
        nii_path = pjoin(ciftify_dir, sub_name, result_dir, nii_dir)
        # loop trial 
        nii_file = pjoin(nii_path, '{0}_Atlas_hp128_s4.dtseries.nii'.format(single_run))
        dt_data = nib.load(nii_file).get_fdata()
        if roi_mat:
            # only save roi
            stim_resp_map[single_run] = np.array(dt_data[:, roi_mat])
        else:
            stim_resp_map[single_run] = np.array(dt_data)
    # transfer to matrix
    task_data = np.dstack(tuple([stim_resp_map[_] for _ in list(stim_resp_map.keys())]))
    task_data = task_data.transpose((2,1,0)).astype(np.float32)
    return task_data

imagenet tsnr computing

In [10]:
# load data
dataset_path = '/nfs/z1/userhome/GongZhengXin/NVP/data_upload/NOD'
ciftify_dir = f'{dataset_path}/derivatives/ciftify'
result_dir = 'results/'
beta_path = './supportfiles/'

task_name = 'imagenet'

n_run = None
sub_names = ['sub-{:02d}'.format(i) for i in range(1,31)]
# Violin plot of the tsnr distribution
all_sub_tsnr_mean = np.zeros((len(sub_names),), dtype=object)

for i, sub_name in enumerate(sub_names):
    # handle special subject
    tsnr_sub_path = pjoin(beta_path, sub_name, f'{sub_name}_imagenet-tsnr.npy')
    if not os.path.exists(pjoin(beta_path, sub_name)):
        os.makedirs(pjoin(beta_path, sub_name))
    if not os.path.exists(tsnr_sub_path):
        dtseries_sum = get_task_time_series(sub_name, ciftify_dir, result_dir, task_name)
        tsnr_sum = np.zeros_like(dtseries_sum[:,:,0])
        n_run = dtseries_sum.shape[0]   
        for run in range(n_run):
            run_data = dtseries_sum[run,:,:]
            tsnr_sum[run, :] = tsnr(run_data)
        tsnr_sub = tsnr_sum.mean(axis=1)
        np.save(tsnr_sub_path, tsnr_sub)
    else:
        tsnr_sub = np.load(tsnr_sub_path)
    # save in data
    all_sub_tsnr_mean[i] = tsnr_sub
np.save(pjoin(beta_path, f'sub-{task_name}_tsnr_mean.npy'), all_sub_tsnr_mean)

# # save tsnr map
# result_path = './supportfiles'

# sub_names = sorted([i for i in os.listdir(beta_path) if i.startswith('sub-')]) 
# tsnr_merge = np.zeros((30, 91282))
# for idx,sub_name in enumerate(sub_names):
#     tsnr_merge[idx] = np.load(pjoin(beta_path, sub_name, f'{sub_name}_imagenet-tsnr.npy'))
# tsnr_map = tsnr_merge.mean(axis=0)
# save_ciftifile(tsnr_map, pjoin(result_path, 'tsnr_map_imagenet.dtseries.nii'))


COCO tsnr computing

In [29]:
# load data
dataset_path = '/nfs/z1/userhome/GongZhengXin/NVP/data_upload/NOD'
ciftify_dir = f'{dataset_path}/derivatives/ciftify'
result_dir = 'results/'
beta_path = './supportfiles/'

task_name = 'coco'

sub_names = sorted(['sub-{:02d}'.format(i) for i in range(1,10)]) 

all_sub_tsnr_mean = np.zeros((len(sub_names),), dtype=object)

for i, sub_name in enumerate(sub_names):
    # handle special subject
    tsnr_sub_path = pjoin(beta_path, sub_name, f'{sub_name}_coco-tsnr.npy')
    if not os.path.exists(tsnr_sub_path):
        dtseries_sum = get_task_time_series(sub_name, ciftify_dir, result_dir, task_name)
        tsnr_sum = np.zeros_like(dtseries_sum[:,:,0])
        n_run = dtseries_sum.shape[0]   
        for run in range(n_run):
            run_data = dtseries_sum[run,:,:]
            tsnr_sum[run, :] = tsnr(run_data)
        tsnr_sub = tsnr_sum.mean(axis=1)
        np.save(tsnr_sub_path, tsnr_sub)
    else:
        tsnr_sub = np.load(tsnr_sub_path)
    # concatenate data
    all_sub_tsnr_mean[i] = tsnr_sub

np.save(pjoin(beta_path, f'sub-{task_name}_tsnr_mean.npy'), all_sub_tsnr_mean)

# # save tsnr map
# result_path = './supportfiles'

# tsnr_merge = np.zeros((len(sub_names), 91282))
# for idx,sub_name in enumerate(sub_names):
#     tsnr_merge[idx] = np.load(pjoin(beta_path, sub_name, f'{sub_name}_coco-tsnr.npy'))
# tsnr_map = tsnr_merge.mean(axis=0)
# save_ciftifile(tsnr_map, pjoin(result_path, 'tsnr_map_coco.dtseries.nii'))


For localizer data

In [28]:
# load data
dataset_path = '/nfs/z1/userhome/GongZhengXin/NVP/data_upload/NOD'
ciftify_dir = f'{dataset_path}/derivatives/ciftify'
result_dir = 'results/'
beta_path = './supportfiles/'

task_name = 'floc'

sub_names = sorted(['sub-{:02d}'.format(i) for i in range(1,10)]) 

all_sub_tsnr_mean = np.zeros((len(sub_names),), dtype=object)

for i, sub_name in enumerate(sub_names):
    # handle special subject
    tsnr_sub_path = pjoin(beta_path, sub_name, f'{sub_name}_localizer-tsnr.npy')
    if not os.path.exists(tsnr_sub_path):
        dtseries_sum = get_task_time_series(sub_name, ciftify_dir, result_dir, task_name)
        tsnr_sum = np.zeros_like(dtseries_sum[:,:,0])
        n_run = dtseries_sum.shape[0]   
        for run in range(n_run):
            run_data = dtseries_sum[run,:,:]
            tsnr_sum[run, :] = tsnr(run_data)
        tsnr_sub = tsnr_sum.mean(axis=1)
        np.save(tsnr_sub_path, tsnr_sub)
    else:
        tsnr_sub = np.load(tsnr_sub_path)
    # collect data
    all_sub_tsnr_mean[i] = tsnr_sub
np.save(pjoin(beta_path, f'sub-{task_name}_tsnr_mean.npy'), all_sub_tsnr_mean)

# # save tsnr map
# result_path = './supportfiles'
# tsnr_merge = np.zeros((len(sub_names), 91282))
# for idx,sub_name in enumerate(sub_names):
#     tsnr_merge[idx] = np.load(pjoin(beta_path, sub_name, f'{sub_name}_localizer-tsnr.npy'))
# tsnr_map = tsnr_merge.mean(axis=0)
# save_ciftifile(tsnr_map, pjoin(result_path, 'tsnr_map_localizer.dtseries.nii'))


For retinotopy data


In [27]:
# load data
dataset_path = '/nfs/z1/userhome/GongZhengXin/NVP/data_upload/NOD'
ciftify_dir = f'{dataset_path}/derivatives/ciftify'
result_dir = 'results/'
beta_path = './supportfiles/'

task_name = 'prf'

sub_names = sorted(['sub-{:02d}'.format(i) for i in range(1,10)]) 

all_sub_tsnr_mean = np.zeros((len(sub_names),), dtype=object)

for i, sub_name in enumerate(sub_names):
    # handle special subject
    tsnr_sub_path = pjoin(beta_path, sub_name, f'{sub_name}_retinotopy-tsnr.npy')
    if not os.path.exists(tsnr_sub_path):
        dtseries_sum = get_task_time_series(sub_name, ciftify_dir, result_dir, task_name)
        tsnr_sum = np.zeros_like(dtseries_sum[:,:,0])
        n_run = dtseries_sum.shape[0]   
        for run in range(n_run):
            run_data = dtseries_sum[run,:,:]
            tsnr_sum[run, :] = tsnr(run_data)
        tsnr_sub = tsnr_sum.mean(axis=1)
        np.save(tsnr_sub_path, tsnr_sub)
    else:
        tsnr_sub = np.load(tsnr_sub_path)
    # collect data
    all_sub_tsnr_mean[i] = tsnr_sub
# save
np.save(pjoin(beta_path, f'sub-{task_name}_tsnr_mean.npy'), all_sub_tsnr_mean)

# # save tsnr map
# result_path = './supportfiles'
# tsnr_merge = np.zeros((len(sub_names), 91282))
# for idx,sub_name in enumerate(sub_names):
#     tsnr_merge[idx] = np.load(pjoin(beta_path, sub_name, f'{sub_name}_retinotopy-tsnr.npy'))
# tsnr_map = tsnr_merge.mean(axis=0)
# save_ciftifile(tsnr_map, pjoin(result_path, 'tsnr_map_retinotopy.dtseries.nii'))
